In [27]:
import numpy as np
import pandas as pd
import math
from scipy.stats import norm
from scipy.optimize import fsolve
from numpy.linalg import inv
import scipy.integrate
from numpy.linalg import inv

In [5]:
def p1(X,mu1):
    return norm.pdf(X,mu1,1)
def p2(X,mu2):
    return norm.pdf(X,mu2,1)
def f(X,lamda,mu1,mu2):
    return (lamda*p1(X,mu1)+(1-lamda)*p2(X,mu2))

In [6]:
N=1000
alpha=0.05
z=norm.ppf(1-alpha/2)
pi=math.pi
F_inverse=[]
H_inverse=[]
lamda_MLE=[]
mu1_MLE=[]
mu2_MLE=[]
while len(F_inverse)<N:
    n=200
    X=[]
    U=np.random.uniform(0,1,n)
    for i in range(n):
        X.append((fsolve(lambda x: 0.5*norm.cdf(x)+0.5*norm.cdf(x-1)-U[i], 0)[0]))
    def ML(paramlist):
        lamda,mu1,mu2=paramlist[0],paramlist[1],paramlist[2]
        s1=0
        s2=0
        s3=0
        for i in range(n):
            s1=s1-(p1(X[i],mu1)-p2(X[i],mu2))/(f(X[i],lamda,mu1,mu2))
            s2=s2-lamda*p1(X[i],mu1)*(X[i]-mu1)/(f(X[i],lamda,mu1,mu2))
            s3=s3-(1-lamda)*p2(X[i],mu2)*(X[i]-mu2)/(f(X[i],lamda,mu1,mu2))
        return s1,s2,s3
    lamda_hat,mu1_hat,mu2_hat=fsolve(ML,[0.5,0,1])
    if lamda_hat>0.0001 and lamda_hat<0.9999:
        lamda_MLE.append(lamda_hat)
        mu1_MLE.append(mu1_hat)
        mu2_MLE.append(mu2_hat)
    else:
        continue
    h11=0
    h12=0
    h13=0
    h22=0
    h23=0
    h33=0
    for i in range(len(X)):
        h11=h11+(1/n)*(p1(X[i],mu1_hat)-p2(X[i],mu2_hat))**2/(f(X[i],lamda_hat,mu1_hat,mu2_hat))**2
        h12=h12-(1/n)*(p1(X[i],mu1_hat)*(X[i]-mu1_hat)*f(X[i],lamda_hat,mu1_hat,mu2_hat)-p1(X[i],mu1_hat)*(X[i]-mu1_hat)*lamda_hat*(p1(X[i],mu1_hat)-p2(X[i],mu2_hat)))/(f(X[i],lamda_hat,mu1_hat,mu2_hat))**2
        h13=h13-(1/n)*(-p2(X[i],mu2_hat)*(X[i]-mu2_hat)*f(X[i],lamda_hat,mu1_hat,mu2_hat)-p2(X[i],mu2_hat)*(X[i]-mu2_hat)*(1-lamda_hat)*(p1(X[i],mu1_hat)-p2(X[i],mu2_hat)))/(f(X[i],lamda_hat,mu1_hat,mu2_hat))**2
        h22=h22-(1/n)*(lamda_hat*p1(X[i],mu1_hat)*(-1+(X[i]-mu1_hat)**2)*f(X[i],lamda_hat,mu1_hat,mu2_hat)-(p1(X[i],mu1_hat)*(X[i]-mu1_hat)*lamda_hat)**2)/(f(X[i],lamda_hat,mu1_hat,mu2_hat))**2
        h33=h33-(1/n)*((1-lamda_hat)*p2(X[i],mu2_hat)*(-1+(X[i]-mu2_hat)**2)*f(X[i],lamda_hat,mu1_hat,mu2_hat)-(p2(X[i],mu2_hat)*(X[i]-mu2_hat)*(1-lamda_hat))**2)/(f(X[i],lamda_hat,mu1_hat,mu2_hat))**2
        h23=h23+(1/n)*(p1(X[i],mu1_hat)*(X[i]-mu1_hat)*lamda_hat*p2(X[i],mu2_hat)*(X[i]-mu2_hat)*(1-lamda_hat))/(f(X[i],lamda_hat,mu1_hat,mu2_hat))**2
    H=[[h11,h12,h13],[h12,h22,h23],[h13,h23,h33]]
    a11= lambda x:(p1(x,mu1_hat)-p2(x,mu2_hat))**2/f(x,lamda_hat,mu1_hat,mu2_hat)
    a22= lambda x:(lamda_hat*(x-mu1_hat)*p1(x,mu1_hat))**2/f(x,lamda_hat,mu1_hat,mu2_hat)
    a33= lambda x:((1-lamda_hat)*(x-mu2_hat)*p2(x,mu2_hat))**2/f(x,lamda_hat,mu1_hat,mu2_hat)
    a12= lambda x:(p1(x,mu1_hat)-p2(x,mu2_hat))*(lamda_hat*(x-mu1_hat)*p1(x,mu1_hat))/f(x,lamda_hat,mu1_hat,mu2_hat)
    a13= lambda x:(p1(x,mu1_hat)-p2(x,mu2_hat))*((1-lamda_hat)*(x-mu2_hat)*p2(x,mu2_hat))/f(x,lamda_hat,mu1_hat,mu2_hat)
    a23 = lambda x:(lamda_hat*(x-mu1_hat)*p1(x,mu1_hat))*((1-lamda_hat)*(x-mu2_hat)*p2(x,mu2_hat))/f(x,lamda_hat,mu1_hat,mu2_hat)
    f11 = scipy.integrate.quad(a11, -35, 35)[0]
    f22 = scipy.integrate.quad(a22, -35, 35)[0]
    f33 = scipy.integrate.quad(a33, -35, 35)[0]
    f12 = scipy.integrate.quad(a12, -35, 35)[0]
    f13 = scipy.integrate.quad(a13, -35, 35)[0]
    f23 = scipy.integrate.quad(a23, -35, 35)[0]
    F=[[f11,f12,f13],[f12,f22,f23],[f13,f23,f33]]
    if 10000>inv(H)[0][0]>0 and 10000>inv(H)[1][1]>0 and 10000>inv(H)[2][2]>0 and 10000>inv(F)[0][0]>0 and 10000>inv(F)[1][1]>0 and 10000>inv(F)[2][2]>0:
        F_inverse.append(inv(F))
        H_inverse.append(inv(H))
    print(len(F_inverse))

1
2
3
4
5
6


C:\Users\jsh98\anaconda3\lib\site-packages\scipy\optimize\minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


7
8
9
10
11
12
13
14
15
16
17
18
19
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
46
47
48
48
48
49
50
50
51
52
53
53
54
55
56
57
57
58
58
59
60
61
62
63
64
65
66
67
68
69
69
70
71
72
73
73
74
75
76
76
77
78
79
80
81
82
83
84
84
85
86
87
88
89
90
90
91
92
93
94
95
95
96
97
98


C:\Users\jsh98\anaconda3\lib\site-packages\scipy\optimize\minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


98
99
100
101
102
103
104
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
119
120
121
122
123
124
125
126
126
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
142
143
144
145
146
147
148
148
149
150
151
152
153
153
154
155
156
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
172
173
174
175
176
177
178
179
180
181
182
183
184
184
184
185
186
187
188
189
190
191
192
192
193
194
195
196
197
198
199
200
201
201
202
203
204
205
205
205
206
207
208
208
209
210
211
212
213
214
215
216
217
218
219
220
221
221
222
223
224
225
226
227
228
229
230
231
232
233
233
234
235
236
237
238
239
240
241
242
242
243
244
245
246
247
248
248
249
250
250
251
252
253
254
255
255
256
256
257
258
259
260
261
262
263
263
264
265
266
267
267
268
269
270
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
290
291
292
293
294
295
296
296
297
298
299
300
301
302
303
303
304
305
306
307
308
309
309
310
311
312
313
314
315
316
317
31

In [7]:
mu1_MLE

[0.2446267205722508,
 0.005712115080656825,
 0.04706040282415914,
 -0.10423252600799676,
 -0.45924396416469315,
 0.14915022810300205,
 -0.3882268310009866,
 -0.042921830959574823,
 0.14721421940680957,
 -0.24001423260786267,
 -0.526543580827392,
 -0.17107411276015616,
 0.25603560082641125,
 -0.5339071113805849,
 -0.7379595140405227,
 0.28329918753019995,
 -0.38781293401624284,
 -0.14645753617692717,
 -0.1448349523202305,
 -0.1092653421446162,
 -0.30418898152658097,
 0.3517765916511281,
 0.06184833472600095,
 -0.24790307541132064,
 -0.32232472811619994,
 -0.4649444304062178,
 0.01608834213835351,
 -0.6582173427570774,
 -0.1455278298523153,
 0.11519438016172057,
 0.01882046551593714,
 0.18953592627559201,
 -0.09284170296602817,
 -0.3622993168263522,
 -0.19310160802685394,
 0.1938491357965614,
 0.15710508315025584,
 0.0826199222311506,
 -0.04695708572066439,
 -1.3195057061090902,
 -0.5519824704664679,
 -0.3286037264511379,
 -0.20683902654664174,
 0.23026839512048966,
 0.1908123839185144,


In [8]:
mu2_MLE

[1.5331839508461953,
 1.4862326014729976,
 1.3254134868912875,
 0.910028742233217,
 0.8814950028587131,
 1.0639789957514414,
 0.699251059915547,
 1.4421873854952763,
 1.1837128996275457,
 1.2079737852591537,
 0.7712335643538849,
 0.8534186187131081,
 0.9158827252229033,
 0.5497663069105224,
 0.8988728706800667,
 1.0928801942028799,
 0.8952663181726287,
 1.2864616475977722,
 1.084373851843829,
 0.6746308673509143,
 0.9288135339698979,
 1.603364225739218,
 1.239710912584385,
 0.7259953781014211,
 0.926701802666226,
 0.90873009911059,
 0.9700477603290402,
 0.8465425080105307,
 1.2496904053468252,
 1.5257482748775824,
 0.6904749024488152,
 1.726545922829187,
 0.9229488266424776,
 1.1739783526623002,
 1.3994619361950251,
 1.2915647797561578,
 1.1095124892395942,
 1.1621061789715417,
 1.1600606067306045,
 0.5658974227600654,
 0.86316250293689,
 0.7541158856368833,
 1.1138291046662294,
 1.3394295028857304,
 0.9885569785845206,
 1.1027585011182277,
 0.965064072642859,
 1.0713868743237318,
 1.6

In [9]:
lamda_MLE

[0.7301422627812038,
 0.6726036256945278,
 0.5825636902597698,
 0.41330719014719897,
 0.29251966100205307,
 0.6127975063831903,
 0.1688091208106329,
 0.654459997415208,
 0.6407043290980136,
 0.46021097057011157,
 0.3012499821848739,
 0.3391287053874252,
 0.6088861393378512,
 0.19625657319869153,
 0.2538965092487658,
 0.7002285634820543,
 0.283546858620596,
 0.6542882581857984,
 0.4120985864707454,
 0.11524140671433449,
 0.3507839077841202,
 0.8483200887151119,
 0.6661527668605998,
 0.3019522960409117,
 0.34881921284276984,
 0.39952971478512606,
 0.6197649210844438,
 0.29420481126040554,
 0.4820734230091751,
 0.6614170571643821,
 0.22402604745247293,
 0.8532631354255205,
 0.47687227160539775,
 0.44688347549750196,
 0.5904951664458538,
 0.7112821061280384,
 0.5232353266237554,
 0.5239071628711633,
 0.5580243013171088,
 0.0557316549901906,
 0.21343899731583107,
 0.25561789502397037,
 0.3249518779816637,
 0.7698453246941793,
 0.35557254893878987,
 0.70197528409038,
 0.5534547801790298,
 0.

In [10]:
v11=np.cov(lamda_MLE,mu1_MLE)[0][0]*200
v12=np.cov(lamda_MLE,mu1_MLE)[0][1]*200
v22=np.cov(lamda_MLE,mu1_MLE)[1][1]*200
v13=np.cov(lamda_MLE,mu2_MLE)[0][1]*200
v33=np.cov(lamda_MLE,mu2_MLE)[1][1]*200
v23=np.cov(mu1_MLE,mu2_MLE)[0][1]*200

In [11]:
V=[[v11,v12,v13],[v12,v22,v23],[v13,v23,v33]]

In [12]:
V

[[11.871473797184724, 15.856733955462943, 15.43907124695684],
 [15.856733955462943, 29.968793434555813, 17.93217825581212],
 [15.43907124695684, 17.93217825581212, 28.47630806639775]]

In [13]:
from numpy import linalg as LA
dF=[]
dH=[]
for i in range(len(F_inverse)):
    dF.append(LA.norm(F_inverse[i]-V))
    dH.append(LA.norm(H_inverse[i]-V))
mF=np.argsort(dF)[len(dF)//2]
mH=np.argsort(dH)[len(dH)//2]

In [14]:
F_inverse[mF]

array([[ 1.2356266 ,  1.42189404,  7.30641163],
       [ 1.42189404,  3.01219003,  8.49149347],
       [ 7.30641163,  8.49149347, 60.81312473]])

In [15]:
H_inverse[mH]

array([[ 8.72568768,  7.50674252, 21.07044315],
       [ 7.50674252,  8.29334134, 16.86170996],
       [21.07044315, 16.86170996, 62.56260208]])

In [16]:
gh11=[]
gf11=[]
gh22=[]
gf22=[]
gh33=[]
gf33=[]
for i in range(len(F_inverse)):
    gh11.append(2*norm.cdf(z*(H_inverse[i][0][0]/v11)**0.5)-1)
    gf11.append(2*norm.cdf(z*(F_inverse[i][0][0]/v11)**0.5)-1)
    gh22.append(2*norm.cdf(z*(H_inverse[i][1][1]/v22)**0.5)-1)
    gf22.append(2*norm.cdf(z*(F_inverse[i][1][1]/v22)**0.5)-1)
    gh33.append(2*norm.cdf(z*(H_inverse[i][2][2]/v33)**0.5)-1)
    gf33.append(2*norm.cdf(z*(F_inverse[i][2][2]/v33)**0.5)-1)

In [17]:
MSEh11=np.mean((gh11-np.asarray(0.95))**2)
MSEf11=np.mean((gf11-np.asarray(0.95))**2)
print(MSEh11)
print(MSEf11)
print(MSEh11/MSEf11)

0.02866571149406056
0.02648350122164799
1.0823988586006446


In [18]:
MSEh22=np.mean((gh22-np.asarray(0.95))**2)
MSEf22=np.mean((gf22-np.asarray(0.95))**2)
print(MSEh22)
print(MSEf22)
print(MSEh22/MSEf22)

0.029601640953493675
0.025156920905914017
1.1766798116590957


In [19]:
MSEh33=np.mean((gh33-np.asarray(0.95))**2)
MSEf33=np.mean((gf33-np.asarray(0.95))**2)
print(MSEh33)
print(MSEf33)
print(MSEh33/MSEf33)

0.027997165361393128
0.024889336851044018
1.1248658623951544


In [20]:
chi_F=[]
chi_H=[]
chi_V=[]
for i in range(1000):
    theta=[[lamda_MLE[i]-0.5,mu1_MLE[i]-0.0,mu2_MLE[i]-1]]
    chi_V.append(np.dot(theta,np.dot(n*inv(V),np.transpose(theta))))
    chi_H.append(np.dot(theta,np.dot(n*inv(H_inverse[i]),np.transpose(theta))))
    chi_F.append(np.dot(theta,np.dot(n*inv(F_inverse[i]),np.transpose(theta))))

In [21]:
np.mean(chi_V)

3.3571585514980335

In [22]:
np.median(chi_V)

2.084522699363716

In [23]:
np.mean(chi_F)

12.628656778587589

In [24]:
np.mean(chi_H)

12.714026948933189

In [25]:
np.median(chi_F)

4.6052512760357

In [26]:
np.median(chi_H)

4.711355560813351